In [1]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier

In [2]:
import keras
import pandas as pd
import numpy as np
import random
import math
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Input, Activation, Flatten, LeakyReLU, Add, GaussianNoise
from keras.layers import LSTM, Embedding, Conv1D, GlobalAveragePooling1D, ZeroPadding1D, MaxPooling1D, GRU, UpSampling1D
from keras.layers import Conv2D, GlobalAveragePooling2D, ZeroPadding2D, MaxPooling2D, UpSampling2D
from keras.layers.merge import concatenate
from keras.models import Model, Sequential, load_model, model_from_json
from keras.layers.normalization import BatchNormalization
from keras.backend import clear_session
from keras import regularizers
from keras.utils import plot_model
from IPython.display import Image
from keras import backend as K
import tensorflow as tf

import csv
import os
from keras.utils import to_categorical
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
Distance = 'cosine'
Train = pd.read_csv(f'.\data\Train_v4_{Distance}_with_handcrafted_features1_2.csv', index_col = 0)
Label = Train['Loan_Status']
Train = Train.drop('Loan_Status',axis = 1)
Test = pd.read_csv(f'.\data\Test_v4_{Distance}_with_handcrafted_features1_2.csv', index_col = 0)

In [4]:
print(Train.shape)
Train

(614, 18)


,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,feature1,feature2
Loan_ID,,,,,,,,,,,,,,,,,,
LP001002,0.0,0.0,1.0,0.0,0.072210,0.000000,0.165714,0.750,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.193917,0.246538
LP001003,0.0,1.0,1.0,0.0,0.056580,0.036192,0.182857,0.750,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.205476,0.236743
LP001005,0.0,1.0,1.0,1.0,0.037037,0.000000,0.094286,0.750,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.215111,0.480667
LP001006,0.0,1.0,0.0,0.0,0.031889,0.056592,0.171429,0.750,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.237469,0.291844
LP001008,0.0,0.0,1.0,0.0,0.074074,0.000000,0.201429,0.750,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.229778,0.240333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,1.0,0.0,1.0,0.0,0.035802,0.000000,0.101429,0.750,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.239387,0.497241
LP002979,0.0,1.0,1.0,0.0,0.050691,0.000000,0.057143,0.375,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.095254,0.175597
LP002983,0.0,1.0,1.0,0.0,0.099654,0.005760,0.361429,0.750,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.297615,0.173484


In [5]:
print(Test.shape)
Test.describe()

(367, 18)


,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,feature1,feature2
count,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000
mean,0.196185,0.634877,0.771117,0.103542,0.059328,0.037670,0.195232,0.712511,0.833787,0.561308,0.163488,0.166213,0.108992,0.302452,0.316076,0.381471,0.238748,0.273298
std,0.397652,0.482122,0.420687,0.305082,0.060626,0.056021,0.088366,0.138158,0.372780,0.496905,0.370315,0.372780,0.312054,0.459947,0.465578,0.486411,0.091139,0.129595
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018624,0.004414
25%,0.000000,0.000000,1.000000,0.000000,0.035358,0.000000,0.143571,0.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190907,0.190703
50%,0.000000,1.000000,1.000000,0.000000,0.046741,0.024600,0.178571,0.750000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.237313,0.262612
75%,0.000000,1.000000,1.000000,0.000000,0.062469,0.058332,0.225714,0.750000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.280793,0.339534
max,1.000000,1.000000,1.000000,1.000000,0.895420,0.575995,0.785714,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.849985


# SVM

In [20]:
Gamma = 0.1
C = 1
clf = SVC(gamma= Gamma, C= C, class_weight='balanced')
clf.fit(Train.values, Label.values) 
OUT = clf.predict(Test.values)
print(OUT)
print(len(OUT))
OUT = [str(i) for i in OUT]

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
367


In [21]:
ANS = pd.DataFrame({'Loan_ID':Test.index, 'Loan_Status':OUT}).replace({'1':'Y', '0':'N'})

In [22]:
ANS.to_csv(f'.\data\SVM_{Distance}_C_'+str(C)+'_Gamma_'+str(Gamma)+'_V4.csv',index = 0)

In [47]:
ANS

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
...,...,...
362,LP002971,N
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y


# MLP

In [126]:
L = 5
hidden = 1
clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(L,hidden),max_iter=1000, random_state=1,learning_rate = 'adaptive')
clf.fit(Train.values, Label.values) 
OUT = clf.predict(Test.values)
print(OUT)
print(len(OUT))
OUT = [str(i) for i in OUT]

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
367


In [127]:
ANS = pd.DataFrame({'Loan_ID':Test.index, 'Loan_Status':OUT}).replace({'1':'Y', '0':'N'})

In [128]:
ANS.to_csv('.\data\MLP_L_'+str(L)+ '_hidden_'+ str(hidden) +'_V2.csv',index = 0)

# using Keras

In [6]:
#compute imbalancing
SUM_1 = np.sum(Label)
class_weights = [SUM_1/Label.shape[0], (Label.shape[0]-SUM_1)/Label.shape[0]]
print(class_weights)

[0.6872964169381107, 0.3127035830618892]


In [7]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(Label.values),Label.values)
print(class_weights)

[1.59895833 0.72748815]


In [10]:
input_size = 4
output_size = 1
L2 = 7e-4
L_rate = 0.01
OUT = []
# for i in range(10):
INPUT =Input(shape = (Train.shape[1],))
x = Dense(input_size, kernel_regularizer = regularizers.l2( L2))(INPUT)
x = Activation('relu')(x)
x = Dense(output_size, kernel_regularizer = regularizers.l2( L2))(x)
out = Activation('sigmoid')(x)

model = Model(inputs=INPUT, outputs=out)
model.summary()
MY_LR = keras.callbacks.ReduceLROnPlateau(monitor='acc', factor=0.5, patience=40, mode='auto', min_delta=0.0001)
MY_ES = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=100,\
                                      verbose=2, mode='auto', restore_best_weights=True)
adam=Adam(lr=L_rate, epsilon=1e-8, decay=0)
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
model.fit(Train.values, Label.values, batch_size=256, epochs=1000, verbose=2,\
          class_weight = class_weights,validation_split = 0.2, callbacks = [MY_LR,MY_ES])
OUT = model.predict(Test.values)

print(OUT)
# print(len(OUT))
#OUT = [str(i) for i in OUT]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 18)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 76        
_________________________________________________________________
activation_3 (Activation)    (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


W0907 22:43:27.188750 10620 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 491 samples, validate on 123 samples
Epoch 1/1000
 - 3s - loss: 0.6062 - acc: 0.6884 - val_loss: 0.6424 - val_acc: 0.6992
Epoch 2/1000
 - 0s - loss: 0.5914 - acc: 0.6884 - val_loss: 0.6374 - val_acc: 0.6829
Epoch 3/1000
 - 0s - loss: 0.5869 - acc: 0.6925 - val_loss: 0.6324 - val_acc: 0.6911
Epoch 4/1000
 - 0s - loss: 0.5812 - acc: 0.6986 - val_loss: 0.6266 - val_acc: 0.6911
Epoch 5/1000
 - 0s - loss: 0.5751 - acc: 0.6986 - val_loss: 0.6215 - val_acc: 0.6911
Epoch 6/1000
 - 0s - loss: 0.5702 - acc: 0.6945 - val_loss: 0.6178 - val_acc: 0.6748
Epoch 7/1000
 - 0s - loss: 0.5648 - acc: 0.6986 - val_loss: 0.6149 - val_acc: 0.6748
Epoch 8/1000
 - 0s - loss: 0.5612 - acc: 0.7026 - val_loss: 0.6116 - val_acc: 0.6829
Epoch 9/1000
 - 0s - loss: 0.5578 - acc: 0.7067 - val_loss: 0.6082 - val_acc: 0.6829
Epoch 10/1000
 - 0s - loss: 0.5537 - acc: 0.7128 - val_loss: 0.6048 - val_acc: 0.6829
Epoch 11/1000
 - 0s - loss: 0.5497 - acc: 0.7210 - val_loss: 0.6024 - val_acc: 0.6829
Epoch 12/1000
 - 

Epoch 96/1000
 - 0s - loss: 0.4479 - acc: 0.8106 - val_loss: 0.5699 - val_acc: 0.7967
Epoch 97/1000
 - 0s - loss: 0.4482 - acc: 0.8106 - val_loss: 0.5702 - val_acc: 0.8049
Epoch 98/1000
 - 0s - loss: 0.4492 - acc: 0.8106 - val_loss: 0.5723 - val_acc: 0.8049
Epoch 99/1000
 - 0s - loss: 0.4479 - acc: 0.8126 - val_loss: 0.5715 - val_acc: 0.7967
Epoch 100/1000
 - 0s - loss: 0.4479 - acc: 0.8126 - val_loss: 0.5707 - val_acc: 0.8049
Epoch 101/1000
 - 0s - loss: 0.4476 - acc: 0.8106 - val_loss: 0.5708 - val_acc: 0.8049
Epoch 102/1000
 - 0s - loss: 0.4477 - acc: 0.8106 - val_loss: 0.5714 - val_acc: 0.8049
Epoch 103/1000
 - 0s - loss: 0.4481 - acc: 0.8106 - val_loss: 0.5725 - val_acc: 0.8049
Epoch 104/1000
 - 0s - loss: 0.4472 - acc: 0.8126 - val_loss: 0.5732 - val_acc: 0.7967
Epoch 105/1000
 - 0s - loss: 0.4473 - acc: 0.8126 - val_loss: 0.5742 - val_acc: 0.7967
Epoch 106/1000
 - 0s - loss: 0.4474 - acc: 0.8126 - val_loss: 0.5736 - val_acc: 0.7967
Epoch 107/1000
 - 0s - loss: 0.4473 - acc: 0.81

In [16]:
keras.backend.clear_session()

In [11]:
threshold = 0.5
OUT2 = []
for i in range(len(OUT)):
    if OUT[i][0] > threshold:
        OUT2.append('1')
    else:
        OUT2.append('0')
print(OUT2)

['1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '1',

In [12]:
ANS = pd.DataFrame({'Loan_ID':Test.index, 'Loan_Status':OUT2}).replace({'1':'Y', '0':'N'})

In [13]:
ANS.to_csv('.\data\MLP_cos_keras_th_'+str(threshold)+'_L_'+str(input_size)+ '_L2_'+ str(L2) +'_V4.csv',index = 0)

In [11]:
Train

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,
LP001002,0.0,0.0,1.0,0.0,0.072210,0.000000,0.214286,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001003,0.0,1.0,1.0,0.0,0.056580,0.036192,0.182857,0.750,1.0,0.0,1.0,0.0,0.0,1,0,0
LP001005,0.0,1.0,1.0,1.0,0.037037,0.000000,0.094286,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001006,0.0,1.0,0.0,0.0,0.031889,0.056592,0.171429,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001008,0.0,0.0,1.0,0.0,0.074074,0.000000,0.201429,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,1.0,0.0,1.0,0.0,0.035802,0.000000,0.101429,0.750,1.0,1.0,0.0,0.0,0.0,1,0,0
LP002979,0.0,1.0,1.0,0.0,0.050691,0.000000,0.057143,0.375,1.0,0.0,0.0,0.0,1.0,1,0,0
LP002983,0.0,1.0,1.0,0.0,0.099654,0.005760,0.361429,0.750,1.0,0.0,1.0,0.0,0.0,0,0,1
